In [ ]:
!pip install farm-haystack[colab]
!pip install farm-haystack transformers
!pip install farm-haystack[inference]
!pip install transformers==4.39.3

  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Usi

In [ ]:
from getpass import getpass
HF_token=getpass("Hugging face token: ")
from haystack import Document
from haystack.nodes import PreProcessor,PromptModel,PromptTemplate,PromptNode
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from haystack.document_stores import InMemoryDocumentStore
from haystack import Pipeline
from haystack.nodes import BM25Retriever
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

Hugging face token: ··········


In [ ]:
def extract_from_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove scripts and styles
        for script_or_style in soup(['script', 'style']):
            script_or_style.extract()

        # Extract and clean the text
        text = soup.get_text(separator='\n')

        # Clean up the text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {e}")
        return ""

def extract_text_from_websites(urls):
    texts = {}
    for url in urls:
        text = extract_from_website(url)
        if text:  # Only append non-empty text
            texts[url] = text
    return texts

urls = [
    "https://stanford-cs324.github.io/winter2022/lectures/introduction/",
    "https://stanford-cs324.github.io/winter2022/lectures/capabilities/",
    "https://stanford-cs324.github.io/winter2022/lectures/harms-1/",
    "https://stanford-cs324.github.io/winter2022/lectures/harms-2/",
    "https://github.com/Hannibal046/Awesome-LLM#milestone-papers"
]

texts = extract_text_from_websites(urls)

# Display the extracted text
for url, text in texts.items():
    print(f"Extracted text from {url}:\n{text[:]}...\n")  # Displaying first 500 characters for brevity


Extracted text from https://stanford-cs324.github.io/winter2022/lectures/introduction/:
Introduction | CS324
Link
Search
Menu
Expand
Document
CS324
Home
Calendar
Lectures
Introduction
Capabilities
Harms I
Harms II
Data
Security
Legality
Modeling
Training
Parallelism
Scaling laws
Selective architectures
Adaptation
Environmental impact
Paper reviews
Paper discussions
Projects
This site uses
Just the Docs
, a documentation theme for Jekyll.
Lectures
Introduction
\[\newcommand{\sV}{\mathcal{V}} \newcommand{\nl}[1]{\textsf{#1}} \newcommand{\generate}[1]{\stackrel{#1}{\rightsquigarrow}}\]
Welcome to CS324! This is a new course on understanding and developing
large language models
.
What is a language model?
A brief history
Why does this course exist?
Structure of this course
What is a language model?
The classic definition of a language model (LM) is a
probability distribution over sequences of tokens
. Suppose we have a
vocabulary
\(\sV\) of a set of tokens. A language model \(p\) assigns e

In [ ]:
docs = [Document(content=text, meta={"url": url}) for url, text in texts.items()]
processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=1000,  # Reduced split length
    split_respect_sentence_boundary=True,
    split_overlap=50,  # Increased split overlap
    language="en",
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
preprocessed_docs = processor.process(docs)

Preprocessing: 100%|██████████| 5/5 [00:00<00:00, 35.38docs/s]


In [ ]:
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(preprocessed_docs)

Updating BM25 representation...: 100%|██████████| 20/20 [00:00<00:00, 1215.18 docs/s]


In [ ]:
retriever = BM25Retriever(document_store, top_k=2)

In [ ]:
def format_prompt(question, context):
  prompt = (
    f"Using only the information contained in the context,"
    f"answer only the question asked without adding suggestions of posible questions and answer exclusively in English. \n"
    f"If the answer cannot be deduced from the context , reply: \"I don't know because it is not relevant to the context\".\n"
    f"context: {context}\n"
    f"question: {question}\n"
    )
  return prompt

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def qa_pipeline(question, retriever):
  try:
    retrieved_docs = retriever.retrieve(query=question)
    context = " ".join([doc.content for doc in retrieved_docs])

    # Limiting input sequence length to 1024 tokens
    max_length = 1024
    formatted_input = format_prompt(question, context)[:max_length]

    input_ids = tokenizer.encode(formatted_input, return_tensors="pt")

    input_ids = input_ids.to(device)

    with torch.no_grad():
      outputs = model.generate(input_ids=input_ids, max_length=1024, num_return_sequences=1, early_stopping=True)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer
  except Exception as e:
    print(f"Error occurred during QA pipeline: {e}")
    return "Error occurred"

In [ ]:
import os

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Now you can run your script or call your function
question = "What is discussed in the harms lectures?"
answer = qa_pipeline(question, retriever)
print(f"Question: {question}")
print(f"Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is discussed in the harms lectures?
Answer: Using only the information contained in the context,answer only the question asked without adding suggestions of posible questions and answer exclusively in English. 
If the answer cannot be deduced from the context, reply: "I don't know because it is not relevant to the context".
context: Harms II | CS324
Link
Search
Menu
Expand
Document
CS324
Home
Calendar
Lectures
Introduction
Capabilities
Harms I
Harms II
Data
Security
Legality
Modeling
Training
Parallelism
Scaling laws
Selective architectures
Adaptation
Environmental impact
Paper reviews
Paper discussions
Projects
This site uses
Just the Docs
, a documentation theme for Jekyll.
Lectures
Harms II
\[\newcommand{\nl}[1]{\textsf{#1}} \newcommand{\generate}[1]{\stackrel{#1}{\rightsquigarrow}}\]
In the last lecture, we started discussing the harms (negative impacts) on
people
who use systems powered by large language models. We call these
behavioral harms
because these are harms